<a href="https://colab.research.google.com/github/Sara300804/BDT/blob/main/EX_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!apt-get update -y
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Hadoop 3.3.6
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz
!mv hadoop-3.3.6 /usr/local/hadoop

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["PATH"] = f"{os.environ['HADOOP_HOME']}/bin:{os.environ['JAVA_HOME']}/bin:" + os.environ["PATH"]

!hadoop version


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,138 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InReleas

In [6]:

!mkdir input_data

with open("input_data/input1.txt", "w") as f:
    f.write("Hello World Bye World\n")

with open("input_data/input2.txt", "w") as f:
    f.write("Hello nanba Goodbye Hadoop\n")

!cat input_data/*


mkdir: cannot create directory ‘input_data’: File exists
Hello World Bye World
Hello nanba Goodbye Hadoop


In [7]:
%%bash
cat > mapper.py <<'EOF'
#!/usr/bin/env python3
import sys

for line in sys.stdin:
    line = line.strip()
    for word in line.split():
        print(f"{word}\t1")
EOF

cat > combiner.py <<'EOF'
#!/usr/bin/env python3
import sys
from collections import defaultdict

counts = defaultdict(int)
for line in sys.stdin:
    word, count = line.strip().split("\t")
    counts[word] += int(count)

for word, count in counts.items():
    print(f"{word}\t{count}")
EOF

cat > reducer.py <<'EOF'
#!/usr/bin/env python3
import sys

current_word = None
current_count = 0

for line in sys.stdin:
    word, count = line.strip().split("\t")
    count = int(count)

    if current_word == word:
        current_count += count
    else:
        if current_word:
            print(f"{current_word}\t{current_count}")
        current_word = word
        current_count = count

if current_word:
    print(f"{current_word}\t{current_count}")
EOF

chmod +x mapper.py combiner.py reducer.py


In [8]:
!hadoop fs -rm -r -f input output

# Put input files into HDFS
!hadoop fs -mkdir -p input
!hadoop fs -put input_data/* input/

# Run Hadoop Streaming
!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
  -input input \
  -output output \
  -mapper mapper.py \
  -combiner combiner.py \
  -reducer reducer.py \
  -file mapper.py \
  -file combiner.py \
  -file reducer.py


2025-11-14 17:36:59,858 INFO Configuration.deprecation: io.bytes.per.checksum is deprecated. Instead, use dfs.bytes-per-checksum
Deleted input
Deleted output
2025-11-14 17:37:03,496 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper.py, combiner.py, reducer.py] [] /tmp/streamjob8246745826307457099.jar tmpDir=null
2025-11-14 17:37:04,015 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-11-14 17:37:04,089 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-11-14 17:37:04,089 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-11-14 17:37:04,107 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-11-14 17:37:04,291 INFO mapred.FileInputFormat: Total input files to process : 2
2025-11-14 17:37:04,307 INFO mapreduce.JobSubmitter: number of splits:2
2025-11-14 17:37:04,466 INFO mapreduce.JobSubmitter: Submitting tokens

In [9]:

!hadoop fs -cat output/part-00000


Bye	1
Goodbye	1
Hadoop	1
Hello	2
World	2
nanba	1
